# Carga Train y Test

In [4]:
import pandas as pd
# Cargar datos de entrenamiento y prueba
train_data = pd.read_excel("../phishing_train.xlsX")
test_data = pd.read_excel("../phishing_test.xlsx")

# Random Forest

In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Separar características (X) y etiquetas (y) en el conjunto de entrenamiento
X_train = train_data.drop(columns=['label'])  # Características de entrenamiento
y_train = train_data['label']  # Etiquetas de entrenamiento

# Entrenar modelo
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Predecir etiquetas para los datos de prueba
y_pred_test = model.predict(test_data)

# Agregar predicciones a los datos de prueba
test_data['predicted_label'] = y_pred_test

# Guardar resultados en un archivo (opcional)
test_data.to_excel("phishing_test_predicted.xlsx", index=False)

print("Predicciones completadas. Resultados guardados en 'phishing_test_predicted.xlsx'.")


ValueError: could not convert string to float: 'ca'

# Gradient Boosting